In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import os

import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import AutoTokenizer

from classes.TwitterDataModule import TwitterDataModule
from classes.Heirarchichal import HeirarchichalNN
from classes.MultiTask import MultiTaskNN
from classes.SingleTask import SingleTaskNN


C:\Users\Arian\.conda\envs\ai-ds-test4\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./config/config.json', 'r') as f:
    config = json.load(f)

print('-------READING TRAINING FILES-------')

train_df = pd.read_csv('../res/preprocessed/train_final.csv')
val_df = pd.read_csv('../res/preprocessed/val_final.csv')
test_df = pd.read_csv('../res/preprocessed/test_final.csv')
print('-------READING TRAINING FILES COMPLETED-------')
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

LABEL_COLUMNS = list(train_df.columns)
LABEL_COLUMNS.remove('text')

TASK1_LABELS = LABEL_COLUMNS[:3]
TASK2_LABELS = LABEL_COLUMNS[3:4]
TASK3_LABELS = LABEL_COLUMNS[4:]


-------READING TRAINING FILES-------
-------READING TRAINING FILES COMPLETED-------


In [3]:


print('-------INITIALIZING TOKENIZER-------')
tokenizer = AutoTokenizer.from_pretrained(config['bert_model_name'])

print('-------CREATING DATA MODULE-------')
data_module = TwitterDataModule(
    train_df,
    val_df,
    test_df,
    tokenizer,
    batch_size=config["batch_size"],
    max_token_len=config["max_token_len"]
)
data_module.setup()
print('-------DATA MODULE CREATED-------')


-------INITIALIZING TOKENIZER-------
-------CREATING DATA MODULE-------
-------DATA MODULE CREATED-------


In [4]:
steps_per_epoch = len(train_df) // config["batch_size"]
total_training_steps = steps_per_epoch * config["n_epochs"]
warmup_steps = total_training_steps // 5

checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss: .2f}',
    save_top_k=2,
    verbose=True,
    monitor="val_loss",
    mode="min",
    every_n_train_steps=100,
)


### TRAIN HIERARCHICHAL

In [5]:
logger = TensorBoardLogger("./logs/lightning_logs",
                           config["heirarchical_logs"])


print('-------INITIALIZING TWITTER NEURAL NET-------')
model = HeirarchichalNN(
    task1_n_classes=len(TASK1_LABELS),
    task2_n_classes=len(TASK2_LABELS),
    task3_n_classes=len(TASK3_LABELS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    bert_model_name=config['bert_model_name']
)
print('-------NEURAL NET INITIALIZED-------')

print('-------TRAINING-------')

trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    gpus=1,
    progress_bar_refresh_rate=config['progress_bar_refresh_rate'],
    num_sanity_val_steps=config['num_sanity_val_steps'],
    fast_dev_run=False,
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    default_root_dir="saved/checkpoints"
)
trainer.fit(model, data_module)

print('-------TRAINING COMPLETE-------')

print('-------EXPORTING MODEL-------')
PATH = './torch_model'
torch.save(model.state_dict(), os.path.join(
    PATH, config["heirarchichal_model"]))
print('-------MODEL EXPORTED SUCCESSFULLY-------')


-------INITIALIZING TWITTER NEURAL NET-------


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Arian\.conda\envs\ai-ds-test4\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setti

-------NEURAL NET INITIALIZED-------
-------TRAINING-------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | bert             | BertModel        | 109 M 
1 | hidden           | Linear           | 590 K 
2 | hidden1          | Linear           | 3.1 K 
3 | hidden2          | Linear           | 1.5 K 
4 | task1_classifier | Linear           | 2.3 K 
5 | task2_classifier | Linear           | 769   
6 | task3_classifier | Linear           | 3.8 K 
7 | criterion_CE     | CrossEntropyLoss | 0     
8 | criterion_BCE    | BCELoss          | 0     
9 | dropout          | Dropout          | 0     
------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
440.337   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/10 [00:01<?, ?it/s]

C:\Users\Arian\.conda\envs\ai-ds-test4\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\Arian\.conda\envs\ai-ds-test4\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 17: 100%|██████████| 16344/16344 [8:32:17<00:00,  1.88s/it, loss=0.952, v_num=0, train_loss=0.651, val_loss=2.010]      

### TRAIN MULTITASK

In [ ]:
logger = TensorBoardLogger("./logs/lightning_logs",
                           config["multitask_logs"])


print('-------INITIALIZING TWITTER NEURAL NET-------')
model =MultiTaskNN(
    task1_n_classes=len(TASK1_LABELS),
    task2_n_classes=len(TASK2_LABELS),
    task3_n_classes=len(TASK3_LABELS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    bert_model_name=config['bert_model_name']
)
print('-------NEURAL NET INITIALIZED-------')

print('-------TRAINING-------')

trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    gpus=1,
    progress_bar_refresh_rate=config['progress_bar_refresh_rate'],
    num_sanity_val_steps=config['num_sanity_val_steps'],
    fast_dev_run=False,
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    default_root_dir="saved/checkpoints"
)
trainer.fit(model, data_module)

print('-------TRAINING COMPLETE-------')

print('-------EXPORTING MODEL-------')
PATH = './torch_model'
torch.save(model.state_dict(), os.path.join(
    PATH, config["multitask_model"]))
print('-------MODEL EXPORTED SUCCESSFULLY-------')


### SINGLE TASK - 1

In [ ]:
from classes.SingleTask import SingleTaskNN
logger = TensorBoardLogger("./logs/lightning_logs",
                           config["singletask_logs1"])


print('-------INITIALIZING TWITTER NEURAL NET-------')
model = SingleTaskNN(
    task_name="task1",
    taskn_classes=len(TASK1_LABELS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    bert_model_name=config['bert_model_name']
)
print('-------NEURAL NET INITIALIZED-------')

print('-------TRAINING-------')

trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    gpus=1,
    progress_bar_refresh_rate=config['progress_bar_refresh_rate'],
    num_sanity_val_steps=config['num_sanity_val_steps'],
    fast_dev_run=False,
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    default_root_dir="saved/checkpoints"
)
trainer.fit(model, data_module)

print('-------TRAINING COMPLETE-------')

print('-------EXPORTING MODEL-------')
PATH = './torch_model'
torch.save(model.state_dict(), os.path.join(
    PATH, config["singletask_model_1"]))
print('-------MODEL EXPORTED SUCCESSFULLY-------')


### SINGLE TASK - 2

In [ ]:
logger = TensorBoardLogger("./logs/lightning_logs",
                           config["singletask_logs2"])


print('-------INITIALIZING TWITTER NEURAL NET-------')
model = SingleTaskNN(
    task_name="task2",
    taskn_classes=len(TASK2_LABELS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    bert_model_name=config['bert_model_name']
)
print('-------NEURAL NET INITIALIZED-------')

print('-------TRAINING-------')

trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    gpus=1,
    progress_bar_refresh_rate=config['progress_bar_refresh_rate'],
    num_sanity_val_steps=config['num_sanity_val_steps'],
    fast_dev_run=False,
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    default_root_dir="saved/checkpoints"
)
trainer.fit(model, data_module)

print('-------TRAINING COMPLETE-------')

print('-------EXPORTING MODEL-------')
PATH = './torch_model'
torch.save(model.state_dict(), os.path.join(
    PATH, config["singletask_model_2"]))
print('-------MODEL EXPORTED SUCCESSFULLY-------')


### SINGLE TASK - 3

In [ ]:
logger = TensorBoardLogger("./logs/lightning_logs",
                           config["singletask_logs3"])


print('-------INITIALIZING TWITTER NEURAL NET-------')
model = SingleTaskNN(
    task_name="task3",
    taskn_classes=len(TASK3_LABELS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    bert_model_name=config['bert_model_name']
)
print('-------NEURAL NET INITIALIZED-------')

print('-------TRAINING-------')

trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    gpus=1,
    progress_bar_refresh_rate=config['progress_bar_refresh_rate'],
    num_sanity_val_steps=config['num_sanity_val_steps'],
    fast_dev_run=False,
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    default_root_dir="saved/checkpoints"
)
trainer.fit(model, data_module)

print('-------TRAINING COMPLETE-------')

print('-------EXPORTING MODEL-------')
PATH = './torch_model'
torch.save(model.state_dict(), os.path.join(
    PATH, config["singletask_model_3"]))
print('-------MODEL EXPORTED SUCCESSFULLY-------')
